In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Concatenate, Embedding
from tensorflow.keras.models import Model

In [ ]:
datapath = "/content/train/part-" + "00000" + "-of-00362.parquet"
x = pd.read_parquet(datapath, engine='pyarrow')
list(x.columns.values)

['product_id',
 'title',
 'description',
 'tags',
 'type',
 'room',
 'craft_type',
 'recipient',
 'material',
 'occasion',
 'holiday',
 'art_subject',
 'style',
 'shape',
 'pattern',
 'bottom_category_id',
 'bottom_category_text',
 'top_category_id',
 'top_category_text',
 'primary_color_id',
 'primary_color_text',
 'secondary_color_id',
 'secondary_color_text',
 'image/encoded',
 'image/width',
 'image/height']

In [ ]:
datapath = "/content/test/part-" + "00000" + "-of-00362.parquet"
x = pd.read_parquet(datapath, engine='pyarrow')
list(x.columns.values)

['product_id',
 'title',
 'description',
 'tags',
 'type',
 'room',
 'craft_type',
 'recipient',
 'material',
 'occasion',
 'holiday',
 'art_subject',
 'style',
 'shape',
 'pattern',
 'image/encoded',
 'image/width',
 'image/height']

In [ ]:
import pandas as pd
datapath = "/content/train/part-" + "00000" + "-of-00362.parquet"
output_file_parquet = pd.read_parquet(datapath)
output = output_file_parquet.to_numpy()
for i in range(1,6):
  padded_string = str(i).zfill(5)
  datapath = "/content/train/part-" + padded_string + "-of-00362.parquet"
  output_file_parquet = pd.read_parquet(datapath)
  output = np.concatenate((output,output_file_parquet.to_numpy()))

print(output.shape)


(4005, 26)


In [ ]:
lst = output[274]
print(*lst, sep='\n')


1421644802
Wedding For Guest, Couple Design Souvenir, Deluxe Gift Box, Personalized Basket, Fresh Nuts Fruit Wooden Bridal Shower, Wedding Announcement
Perfect for any wedding party. Design is made to order and Black color ink to exact detail. All of our wood is sustainably sourced and hand selected in order to provide the best quality signs possible. \nOur custom-printed wood baskets are printed directly onto the wood material using the ink for a permanent finish that doesn&#39;t peel or fade.  FULL digitally printed. \n\nWedding Gift Basket\nDimensions: 9&quot;x9&quot; \nMaterials: Wood \nCondition: New\n\n🎁 Gift-giving made healthy - wedding, say thank you, guest gifts, or hostess gifts!\n🍓The perfect unique healthy gift idea for any occasion- Our premium gourmet food gift basket will be absolutely loved.\n🎀  100% Money-back guarantee – Buy with Confidence Knowing Your Satisfaction is Guaranteed With a Full Refund or Replacement In the Event of Any Issue.\n🎉 Our Premium Fresh Dried 

In [ ]:
# Generate training dataset
# Make "Type" field as binary where physical -> 0 && download -> 1
for i in range(output.shape[0]):
  if output[i,4] == 'physical':
    output[i,4] = 0
  elif output[i,4] == 'download':
    output[i,4] = 1
  else:
    output[i,4] = 2

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Create encoder
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',
                                 unknown_value=-1)
ordinal_encoder.fit(output[:,6].reshape(-1, 1))
x = ordinal_encoder.transform(output[:,6].reshape(-1, 1))
output[:,6] = x.reshape(1,-1)


29.0


In [ ]:
# Create Bag of all tags
tag_words = set(output[0,3].split(','))

for i in range(1,output.shape[0]):
  s = set(output[i,3].split(','))
  tag_words.update(s)

print(len(tag_words))

26070


In [ ]:
lst = output[275]
print(*lst, sep='\n')

1464212913
Salad dressing or salmon glaze - birch syrup Soak will elevate the dish to another whole new level
Birch syrup - sweet and sour. A perfect salad dressing and a salmon glaze.\n\nWOOD WIDE WEB\nA forest is just as endless as cosmos is. An opportunity to escape and get lost in a good way. The forest lives in layers – the trees, the bushes, the berries, mushrooms, moss, roots and more that are all connected through complex systems of nurturing and symbiosis. Sometimes we think we know it, but it turns out to be an eternal abundance to explore. For us forest feels like home, but we’re curious about the parts we’ve haven’t explored yet. The small details and unchartered territories of the wood wide web. This is where our products come from.\n\nSoak Birch Syrup is made of birch sap that is first reduced with the process of reverse osmosis and then boiled down to the final consistency and brix 67.
food,salad,dressing,black,sweet,sour,sauce,culinary,brown,bottle
0

0.0








965
ho

In [ ]:
# X_train = np.asarray(X_train).astype(np.float32)
# X_train = np.zeros((output.shape[0],4+len(tag_words)))
X_train = np.zeros((output.shape[0],4))
for i in [4,6,24,25]:
  num = 0
  X_train[:,num] = output[:,i]
  num = num+1

# for i in range(1,output.shape[0]):
#   num = 0
#   for j in tag_words:
#     s = set(output[i,3].split(','))
#     X_train[i,4+num] = s.issubset({j})
#     num = num+1

filter_indices_Y = [15,17,19,21]
Y_train = np.take(output,filter_indices_Y,1)
Y_train = np.asarray(Y_train).astype(np.float32)
print(X_train.shape)
print(Y_train.shape)

(4005, 4)
(4005, 4)


In [ ]:
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
# Create a sequential model
model = Sequential()

In [ ]:
# Add a dense hidden layer (you can adjust the number of units and other hyperparameters)
model.add(LSTM(64, return_sequences=True, input_shape=(4, 1)))  # Adjust units as needed
model.add(Dropout(0.2))
model.add(LSTM(64, input_shape=(4, 1)))  # Adjust units as needed
model.add(Dropout(0.2))

In [ ]:
# Add the output layer with 4 neurons (linear activation for regression)
model.add(Dense(4, activation='softmax'))  # Output layer with 4 neurons

In [ ]:
# Define the loss function
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 4, 64)             16896     
                                                                 
 dropout_4 (Dropout)         (None, 4, 64)             0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 260       
                                                                 
Total params: 50180 (196.02 KB)
Trainable params: 50180 (196.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, Y_train, epochs=20, batch_size=32)

Epoch 1/20
126/126 [==============================] - 1s 9ms/step - loss: 720.4787 - accuracy: 0.9925
Epoch 2/20
126/126 [==============================] - 1s 7ms/step - loss: 735.4697 - accuracy: 0.9900
Epoch 3/20
126/126 [==============================] - 1s 7ms/step - loss: 801.9778 - accuracy: 0.9873
Epoch 4/20
126/126 [==============================] - 1s 10ms/step - loss: 819.8592 - accuracy: 0.9853
Epoch 5/20
126/126 [==============================] - 1s 11ms/step - loss: 842.6417 - accuracy: 0.9838
Epoch 6/20
126/126 [==============================] - 1s 10ms/step - loss: 930.7356 - accuracy: 0.9785
Epoch 7/20
126/126 [==============================] - 1s 9ms/step - loss: 1046.2983 - accuracy: 0.9623
Epoch 8/20
126/126 [==============================] - 1s 6ms/step - loss: 1089.6016 - accuracy: 0.9625
Epoch 9/20
126/126 [==============================] - 1s 7ms/step - loss: 1144.2405 - accuracy: 0.9548
Epoch 10/20
126/126 [==============================] - 1s 7ms/step - loss: 1

In [ ]:
# test dataset
datapath = "/content/test/part-" + "00000" + "-of-00362.parquet"
output_file_parquet = pd.read_parquet(datapath)
test = output_file_parquet.to_numpy()
for i in range(1,6):
  padded_string = str(i).zfill(5)
  datapath = "/content/test/part-" + padded_string + "-of-00362.parquet"
  output_file_parquet = pd.read_parquet(datapath)
  test = np.concatenate((test,output_file_parquet.to_numpy()))

print(test.shape)

(104, 18)


In [ ]:
lst = test[1]
print(*lst, sep='\n')

1501009290
5 Colour Slatepencils In One Pack Order NOw
5 Colour Slatepencils In One Pack Order NOw\n\nFive Colours Include -\n1] White Thick Slatepencils\n2] Red Thick Slatepencils\n3] Pink Thick Slatepencils\n4] Yellow THick  Slatepencils\n5] Black Thick Slatepencils\n\nOrder now best earthy crunchy pencils\nSuper made from natural stone
slatepencils,clay,nakumatt,edibleclay
physical

drawing & drafting








b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x01A\x02:\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\

In [ ]:
'product_id',
 'title',
 'description',
 'tags',
 'type',
 'room',
 'craft_type',
 'recipient',
 'material',
 'occasion',
 'holiday',
 'art_subject',
 'style',
 'shape',
 'pattern',
 'image/encoded',
 'image/width',
 'image/height'

IndentationError: unexpected indent (<ipython-input-57-89fde7dcae2d>, line 2)

In [ ]:
for i in range(test.shape[0]):
  if test[i,4] == 'physical':
    test[i,4] = 0
  elif test[i,4] == 'download':
    test[i,4] = 1
  else:
    test[i,4] = 2

In [ ]:
x = ordinal_encoder.transform(test[:,6].reshape(-1, 1))
test[:,6] = x.reshape(1,-1)

In [ ]:
print(*test[1],sep='\n')

0.0
-1.0
570.0
321.0


In [ ]:
filter_indices_test = [4,6,16,17]
test = np.take(test,filter_indices_test,1)
test = np.asarray(test).astype(np.float32)

In [ ]:
# Make predictions
# Prepare test input vector
test = np.array(test)
test = np.reshape(test, (test.shape[0], test.shape[1], 1))
predicted_output = model.predict(X_train)

print("Predicted output:", predicted_output)

print(Y_train)



126/126 [==============================] - 1s 6ms/step
Predicted output: [[0.6837076  0.302552   0.00521508 0.00852531]
 [0.6837076  0.302552   0.00521508 0.00852531]
 [0.6837076  0.302552   0.00521508 0.00852531]
 ...
 [0.6837076  0.302552   0.00521508 0.00852531]
 [0.6837076  0.302552   0.00521508 0.00852531]
 [0.6837076  0.302552   0.00521508 0.00852531]]
[[4.260e+02 5.000e+00 2.000e+00 2.000e+00]
 [4.260e+02 5.000e+00 1.100e+01 1.000e+00]
 [4.260e+02 5.000e+00 1.700e+01 1.000e+00]
 ...
 [1.803e+03 1.400e+01 1.200e+01 1.100e+01]
 [1.803e+03 1.400e+01 7.000e+00 1.400e+01]
 [1.803e+03 1.400e+01 1.600e+01 1.700e+01]]
